# Using the RIOT games API

Many APIs available at https://developer.riotgames.com/apis. Most basic is `SUMMONER-V4`

## Types of APIs

### `SUMMONER-V4`

In [55]:
"""
Generate API keys using login details in slack
"""
API_key = "RGAPI-20fd6f2f-f1b5-4049-9178-9291f7f9d239" # paste key here 

Appending this key to the URL 

"https://na1.api.riotgames.com/lol/summoner/v4/summoners/by-name/Doublelift?api_key="

gives the following data

`{"id":"McE7ghJoqLBKaRWqDGjWm9niZf-tiJqo1DoNZuFtD1-uV3s",
 "accountId":"9_BG7jTCi0ZGfrz0bbJuQf4qcdQD_qO9Etwrh_8wm8_WOw",
 "puuid":"uE_qjrHfzSicPneHSjhpV3n8H74hBLK_OJIIBnduoFCocTQf45kAsZtODNtwXtEYkCNSznea2DOLeA",
 "name":"Doublelift",
 "profileIconId":4568,
 "revisionDate":1623977717000,
 "summonerLevel":255}`

Can parse this as a python dictionary if we wanted

In [2]:
import json 
static_data =  '{"id":"McE7ghJoqLBKaRWqDGjWm9niZf-tiJqo1DoNZuFtD1-uV3s",\
 "accountId":"9_BG7jTCi0ZGfrz0bbJuQf4qcdQD_qO9Etwrh_8wm8_WOw",\
 "puuid":"uE_qjrHfzSicPneHSjhpV3n8H74hBLK_OJIIBnduoFCocTQf45kAsZtODNtwXtEYkCNSznea2DOLeA",\
 "name":"Doublelift",\
 "profileIconId":4568,\
 "revisionDate":1623977717000,\
 "summonerLevel":255}'

# parse as dictionary
static_data = json.loads(static_data)
for i, j in static_data.items():
    print(str(i)+" : "+str(j))

profileIconId : 4568
name : Doublelift
puuid : uE_qjrHfzSicPneHSjhpV3n8H74hBLK_OJIIBnduoFCocTQf45kAsZtODNtwXtEYkCNSznea2DOLeA
summonerLevel : 255
accountId : 9_BG7jTCi0ZGfrz0bbJuQf4qcdQD_qO9Etwrh_8wm8_WOw
id : McE7ghJoqLBKaRWqDGjWm9niZf-tiJqo1DoNZuFtD1-uV3s
revisionDate : 1623977717000


Or we could begrudgingly use pandas to pull straight from the url

In [8]:
import pandas as pd
url = "https://na1.api.riotgames.com/lol/summoner/v4/summoners/by-name/Doublelift?api_key="+str(API_key)
df = pd.read_json(url, typ='series')
print(df)

accountId           9_BG7jTCi0ZGfrz0bbJuQf4qcdQD_qO9Etwrh_8wm8_WOw
id                 McE7ghJoqLBKaRWqDGjWm9niZf-tiJqo1DoNZuFtD1-uV3s
name                                                    Doublelift
profileIconId                                                 4568
puuid            uE_qjrHfzSicPneHSjhpV3n8H74hBLK_OJIIBnduoFCocT...
revisionDate                                         1624657074000
summonerLevel                                                  255
dtype: object


This url is the root of all API requests from riot games. 

In [78]:
root = "https://na1.api.riotgames.com/lol" # note the region prefix can vary from the example "na1.api" used here
url = root+"/summoner/v4/summoners/by-name/Doublelift?api_key="+str(API_key)
df = pd.read_json(url, typ='series')
print(df)

accountId           9_BG7jTCi0ZGfrz0bbJuQf4qcdQD_qO9Etwrh_8wm8_WOw
id                 McE7ghJoqLBKaRWqDGjWm9niZf-tiJqo1DoNZuFtD1-uV3s
name                                                    Doublelift
profileIconId                                                 4568
puuid            uE_qjrHfzSicPneHSjhpV3n8H74hBLK_OJIIBnduoFCocT...
revisionDate                                         1624657074000
summonerLevel                                                  255
dtype: object


### collecting match data

Can get large lists of match and player (Summoner) data by requesting from ranked leaderboards

In [76]:
# table of ranked player data
url = "https://euw1.api.riotgames.com/lol/league/v4/entries/RANKED_SOLO_5x5/DIAMOND/I?api_key="+str(API_key)
ranked_players = pd.read_json(url)
print(ranked_players.head()) # summoner IDs of the top 205 ranked players

   freshBlood  hotStreak  inactive                              leagueId  \
0       False      False     False  8cb3ecdd-f866-4372-b20f-ad48f737c60b   
1       False      False     False  b2a1013f-d3f6-472a-893a-b0af8bc00823   
2       False       True     False  36af9f47-0c1c-432a-8a76-4ea50136213b   
3       False       True     False  5a3cec92-cdee-4693-afbc-a381f40242ee   
4       False      False     False  e3f543a7-a957-40a4-8f9c-c93bee137914   

   leaguePoints  losses miniSeries        queueType rank  \
0            30     280        NaN  RANKED_SOLO_5x5    I   
1            43     194        NaN  RANKED_SOLO_5x5    I   
2            67     102        NaN  RANKED_SOLO_5x5    I   
3            27      42        NaN  RANKED_SOLO_5x5    I   
4            75      64        NaN  RANKED_SOLO_5x5    I   

                                         summonerId      summonerName  \
0  LlBWh6eU352yPodpM3lBdhlyMYBcpf-A5LLJ1Zg2qC0Z-7as   ya ne ispravlus   
1  Ou7L3HEAyawldXah_rKqsLbmG2FM6TbyF

Can then use the `summonerName` field to grab `accountId` using the `by-name` endpoint

In [82]:
summoner = ranked_players.summonerName[2] # chose 2 at random
print(summoner)

# pull this summoners player data
url = "https://euw1.api.riotgames.com/lol/summoner/v4/summoners/by-name/"\
        +str(summoner)+"?api_key="+str(API_key)
summoner_params = pd.read_json(url, typ='series')
print(summoner_params)

MiriaelSabathiel
accountId          l3-pQLjHuqYRMuthjRG_6KFVPF-nZZ9bi-KGs8yK4sALQBk
id                xXAIoJV1A7aG1psY1edidTYofou0R5FMiaajudPSZG-aThXc
name                                              MiriaelSabathiel
profileIconId                                                 3790
puuid            WUcUQ4kmbcRU31RnsjTwB6JYDBIIWa_L7RpXKjfyQnpVzo...
revisionDate                                         1624565753000
summonerLevel                                                  172
dtype: object


We can then pull the match history of this player using the `MATCH-V4` API

In [86]:
# pull this summoners match history data
url = "https://euw1.api.riotgames.com/lol/match/v4/matchlists/by-account/"\
        +str(summoner_params.accountId)+"?api_key="+str(API_key)
match_hist = pd.read_json(url, typ='series')

In [96]:
print(len(match_hist.matches)) # total matches available
print(match_hist.matches[1])

100
{u'lane': u'NONE', u'gameId': 5337624784L, u'champion': 10, u'platformId': u'EUW1', u'timestamp': 1624564590676L, u'queue': 420, u'role': u'DUO_SUPPORT', u'season': 13}


In [105]:
# pull this summoners match history data
print(str((match_hist.matches[1])["gameId"]))
url = "https://euw1.api.riotgames.com/lol/match/v5/matches/"+str((match_hist.matches[1])["gameId"])\
        +"?api_key="+str(API_key)
match_ = pd.read_json(url, typ='series')

5337624784


HTTPError: HTTP Error 400: Bad Request